In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np
from tqdm import tqdm

print('PyTorch Version : ', torch.__version__) 
print('GPU : ', torch.cuda.device_count())
GPUNUM = torch.cuda.device_count()
if GPUNUM == 0:
    torch.cuda.is_available = lambda : False
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=2, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1)
        self.fc1 = nn.Linear(2048, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.conv3(x)
        x = F.relu(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output


def client_update(client_model, optimizer, train_loader, epoch=5):
    model.train()
    for e in range(epoch):
        for batch_idx, (data, target) in enumerate(train_loader):
            if GPUNUM == 0:
                data, target = data.cpu(), target.cpu()
            else :
                data, target = data.cuda(), target.cuda()
            # data, target = data.cuda(), target.cuda()
            optimizer.zero_grad()
            output = client_model(data)
            loss = F.nll_loss(output, target)
            loss.backward()
            optimizer.step()
    return loss.item()

def server_aggregate(global_model, client_models):
    global_dict = global_model.state_dict()
    for k in global_dict.keys():
        global_dict[k] = torch.stack([client_models[i].state_dict()[k] for i in range(len(client_models))], 0).mean(0)
    global_model.load_state_dict(global_dict)
    for model in client_models:
        model.load_state_dict(global_model.state_dict())

def test(global_model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            if GPUNUM == 0:
                data, target = data.cpu(), target.cpu()
            else :
                data, target = data.cuda(), target.cuda()            
            # data, target = data.cuda(), target.cuda()
            output = global_model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    acc = correct / len(test_loader.dataset)

    return test_loss, acc

PyTorch Version :  2.2.2
GPU :  0


In [2]:
# IID case: all the clients have images of all the classes

# Hyperparameters

num_clients = 100
num_selected = 10
num_rounds = 5
epochs = 5
batch_size = 32

# Creating decentralized datasets

traindata = datasets.MNIST('./data', train=True, download=True,
                       transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.1307,), (0.3081,))])
                       )
traindata_split = torch.utils.data.random_split(traindata, [int(traindata.data.shape[0] / num_clients) for _ in range(num_clients)])
train_loader = [torch.utils.data.DataLoader(x, batch_size=batch_size, shuffle=True) for x in traindata_split]

test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('./data', train=False, transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.1307,), (0.3081,))])
        ), batch_size=batch_size, shuffle=True)

# Instantiate models and optimizers

if GPUNUM == 0:
    global_model = Net().cpu()
    client_models = [Net().cpu() for _ in range(num_selected)]
else:
    global_model = Net().cuda()
    client_models = [Net().cuda() for _ in range(num_selected)]

# global_model = Net().cuda()
# client_models = [Net().cuda() for _ in range(num_selected)]
for model in client_models:
    model.load_state_dict(global_model.state_dict())

opt = [optim.SGD(model.parameters(), lr=0.1) for model in client_models]

# Runnining FL

for r in range(num_rounds):
    # select random clients
    client_idx = np.random.permutation(num_clients)[:num_selected]

    # client update
    loss = 0
    for i in range(num_selected):
        loss += client_update(client_models[i], opt[i], train_loader[client_idx[i]], epoch=epochs)
    
    # serer aggregate
    server_aggregate(global_model, client_models)
    test_loss, acc = test(global_model, test_loader)
    
    print('%d-th round' % r)
    print('average train loss %0.3g | test loss %0.3g | test acc: %0.3f' % (loss / num_selected, test_loss, acc))

0-th round
average train loss 0.549 | test loss 0.462 | test acc: 0.869
1-th round
average train loss 0.113 | test loss 0.262 | test acc: 0.924
2-th round
average train loss 0.0599 | test loss 0.191 | test acc: 0.945
3-th round
average train loss 0.0575 | test loss 0.161 | test acc: 0.955
4-th round
average train loss 0.043 | test loss 0.145 | test acc: 0.960


In [3]:
# NON-IID case: every client has images of two categories chosen from [0, 1], [2, 3], [4, 5], [6, 7], or [8, 9].

# Hyperparameters

num_clients = 100
num_selected = 5
num_rounds = 50
epochs = 5
batch_size = 32

# Creating decentralized datasets

traindata = datasets.MNIST('./data', train=True, download=True,
                       transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.1307,), (0.3081,))])
                       )
target_labels = torch.stack([traindata.targets == i for i in range(10)])
target_labels_split = []
for i in range(5):
    target_labels_split += torch.split(torch.where(target_labels[(2 * i):(2 * (i + 1))].sum(0))[0], int(60000 / num_clients))
traindata_split = [torch.utils.data.Subset(traindata, tl) for tl in target_labels_split]
train_loader = [torch.utils.data.DataLoader(x, batch_size=batch_size, shuffle=True) for x in traindata_split]

test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('./data', train=False, transform=transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.1307,), (0.3081,))])
        ), batch_size=batch_size, shuffle=True)

# Instantiate models and optimizers
if GPUNUM == 0:
    global_model = Net().cpu()
    client_models = [Net().cpu() for _ in range(num_selected)]
else:
    global_model = Net().cuda()
    client_models = [Net().cuda() for _ in range(num_selected)]
# global_model = Net().cuda()
# client_models = [Net().cuda() for _ in range(num_selected)]
for model in client_models:
    model.load_state_dict(global_model.state_dict())

opt = [optim.SGD(model.parameters(), lr=0.1) for model in client_models]

# Runnining FL

for r in range(num_rounds):
    # select random clients
    client_idx = np.random.permutation(num_clients)[:num_selected]

    # client update
    loss = 0
    for i in range(num_selected):
        loss += client_update(client_models[i], opt[i], train_loader[client_idx[i]], epoch=epochs)
    
    # serer aggregate
    server_aggregate(global_model, client_models)
    test_loss, acc = test(global_model, test_loader)
    
    print('%d-th round' % r)
    print('average train loss %0.3g | test loss %0.3g | test acc: %0.3f' % (loss / num_selected, test_loss, acc))

0-th round
average train loss 0.0318 | test loss 2.8 | test acc: 0.275
1-th round
average train loss 0.0179 | test loss 1.94 | test acc: 0.409
2-th round
average train loss 0.00782 | test loss 1.83 | test acc: 0.428
3-th round
average train loss 0.0245 | test loss 2.09 | test acc: 0.400
4-th round
average train loss 0.00537 | test loss 4.73 | test acc: 0.215
5-th round
average train loss 0.00362 | test loss 3.4 | test acc: 0.189
6-th round
average train loss 0.00404 | test loss 3.32 | test acc: 0.321
7-th round
average train loss 0.0694 | test loss 1.36 | test acc: 0.520
8-th round
average train loss 0.0304 | test loss 1.54 | test acc: 0.528
9-th round
average train loss 0.0158 | test loss 1.14 | test acc: 0.700
10-th round
average train loss 0.00102 | test loss 1.05 | test acc: 0.644
11-th round
average train loss 0.00128 | test loss 1.52 | test acc: 0.510
12-th round
average train loss 0.00128 | test loss 2.16 | test acc: 0.378
13-th round
average train loss 0.000803 | test loss 1.95